<font color="red" size=5><center>RSNA-STR Pulmonary Embolism Detection - 肺塞栓症検知 -</center></font>

In [ ]:
# Fork元
注釈： 本記事は https://www.kaggle.com/nitindatta/pulmonary-embolism-dicom-preprocessing-eda を和訳した物です。

# Introduction 

このノートでは、DICOMファイルとCTスキャンについて学びます。また、seabornとmatplotlibを使って表形式のデータを可視化します。最後に何が追加でできるかを説明します。

### **肺塞栓症(Pulmonary Embolism; PE)とは**
*　肺塞栓症は、血液のかたまり（血栓）や、まれに他の固形物が血液の流れに乗って肺の動脈（肺動脈）に運ばれ、そこをふさいでしまう（塞栓）病気です。
* 肺塞栓症の症状には、息切れ、特に息を吸うときの胸の痛み、血痰などがあります。
* ほとんどの場合、肺塞栓症は、足から移動した血栓によって引き起こされます。（エコノミークラス症候群によっても肺塞栓症は引き起こされます。


<font color="red" size=3>このカーネルが役に立てたなら、Upvoteしていただければ幸いです。</font>

In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import HTML

sns.set_style('darkgrid')
import pydicom
import scipy.ndimage
import gdcm
import imageio
from IPython import display


from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage.morphology import disk, opening, closing
from tqdm import tqdm

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly.graph_objs import *
init_notebook_mode(connected=True) 
from PIL import Image

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from os import listdir, mkdir

In [ ]:
basepath = "../input/rsna-str-pulmonary-embolism-detection/"
listdir(basepath)

In [ ]:
train = pd.read_csv(basepath + "train.csv")
test = pd.read_csv(basepath + "test.csv")

In [ ]:
train.shape, test.shape

In [ ]:
train.head().T

# データ概要


* `StudyInstanceUID` - データ内の各検査の一意のID。
* `SeriesInstanceUID` - 検査中の各シリーズに固有のIDです。
* `SOPInstanceUID` - 検査内の各画像に固有のIDです。
* `pe_present_on_image` - 画像レベルで，画像上にPEが存在するかどうかを示します．
* `negative_exam_for_pe` - 検査レベルで、PEが存在する画像があるかどうかを示します。
* `qa_motion` - 検査で放射線技師がモーションアーチファクトの問題を指摘したかどうかを示します。
* `qa_contrast` - 放射線技師が検査で造影に問題があると指摘したかどうかを示します。
* `flow_artifact` - 参考値
* `rv_lv_ratio_gte_1` - 検査レベル, 検査に含まれる RV/LV 比が >= 1 であるかどうかを示します.
* `rv_lv_ratio_lt_1` - 検査レベル, 検査に含まれるRV/LV比が1未満であるかどうかを示します.
* `leftsided_pe` - 検査中の画像の左側に PE が存在することを示します。
* `chronic_pe` - 検査レベルの PE が慢性的なものであることを示します。
* `true_filling_defect_not_pe` - PE ではない疾患を示します。
* `rightsided_pe` - 試験レベルで、試験中の画像の右側に PE が存在することを示します。
* `acute_and_chronic_pe` - 試験に含まれるPEが急性および慢性の両方であることを示します。
* `central_pe` - 試験の画像の中心部に PE が存在することを示します。
* `indeterminate` - 検査は PE に対して陰性ではないが、QA の問題により試験レベルの最終的なラベルセットを作成できなかったことを示しています。

In [ ]:
print("Number of unique Study instances are", train['StudyInstanceUID'].nunique())
print("Number of unique Series instances are", train['SeriesInstanceUID'].nunique())

📌 研究と系列の両方が同じ数であるため、各研究には1つの系列しかないと推論できます。

NULL値、各列のタイプ、メモリ使用量などのデータについて、いくつかのサニティチェックを行います

### 欠損値は存在するか

In [ ]:
print('Null values in train data:',train.isnull().sum().sum())
print('Null values in test data:',test.isnull().sum().sum())

### 欠損値なし

In [ ]:
train.info()

In [ ]:
test.info()

約240MBのテーブルデータを用いる。

In [ ]:
def load_scans(dcm_path):
    files = listdir(dcm_path)
    f = [pydicom.dcmread(dcm_path + "/" + str(file)) for file in files]
    return f

In [ ]:
example = basepath + "train/" + train.StudyInstanceUID.values[0] +'/'+ train.SeriesInstanceUID.values[0]
file_names = listdir(example)

In [ ]:
scans = load_scans(example)

### dicomデータの例

CTスキャンについて

* CTスキャンは、X線を照射された物体や組織の放射線密度に関する情報を取得します。
* 横方向のスライスは、いくつかの異なる方向から測定を行った後、スキャンを再構成します。
* CTスキャンはすでにHUフォーマットになっています。
* CTスキャンでは約4000個のグレー値が得られるが、それは我々の目では捉えられない。そこで、我々は"windowing"を実行します。
* 水はHU 0、空気は-1000


In [ ]:
scans[0]

In [ ]:
plt.figure(figsize=(12,6))
for n in range(5):
    image = scans[n].pixel_array.flatten()
    rescaled_image = image * scans[n].RescaleSlope + scans[n].RescaleIntercept
    sns.distplot(image.flatten());
plt.title("HU unit distributions for 5 examples");

📌 上のグラフでは、5つの例の画素分布をプロットしています。

次に、CTスキャン画像と合わせて画素配列分布を見ていきます。

## Utility Functions

In [ ]:
# dicom画像のロード
def load_slice(path):
    slices = [pydicom.read_file(path + '/' + s) for s in listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

# HU配列に変換
def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

def resample(image, scan, new_spacing=[1,1,1]):
    spacing = np.array([float(scans_0[0].SliceThickness), 
                        float(scans_0[0].PixelSpacing[0]), 
                        float(scans_0[0].PixelSpacing[0])])


    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

def make_mesh(image, threshold=-300, step_size=1):
    p = image.transpose(2,1,0)
    verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold, step_size=step_size, allow_degenerate=True)
    return verts, faces


def plt_3d(verts, faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
#     ax.set_axis_bgcolor((0.7, 0.7, 0.7))
    ax.set_facecolor((0.7,0.7,0.7))
    plt.show()


In [ ]:
sns.set_style('white')
hu_scans = transform_to_hu(scans)

fig, ax = plt.subplots(1,2,figsize=(15,4))


ax[0].set_title("CT-scan in HU")
ax[0].imshow(hu_scans[0], cmap="plasma")
ax[1].set_title("HU values distribution");
sns.distplot(hu_scans[0].flatten(), ax=ax[1],color='red', kde_kws=dict(lw=2, ls="--",color='blue'));
ax[1].grid(False)

📌 グラフから、空気は約-1000のHUを持ち、次に高いのは水であることから、面積の大部分が空気で満たされていることが推察できます。HUは約0

In [ ]:
first_patient = load_slice('../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf')
first_patient_pixels = transform_to_hu(first_patient)

def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[18,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title(f'slice {ind}')
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='bone')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(first_patient_pixels)

順番に並べて5枚ずつ飛ばして、より多くの種類のスライスを見られるようにしています。

In [ ]:
imageio.mimsave("/tmp/gif.gif", first_patient_pixels, duration=0.1)
display.Image(filename="/tmp/gif.gif", format='png')

In [ ]:
first_patient_scan = '../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf'
scans_0 = load_scans(first_patient_scan)
imgs_after_resamp, spacing = resample(first_patient_pixels, scans_0, [1,1,1])
v, f = make_mesh(imgs_after_resamp, threshold = 350)
plt_3d(v, f)

In [ ]:
im_path = []
train_path = '../input/rsna-str-pulmonary-embolism-detection/train/'
for i in listdir(train_path): 
    for j in listdir(train_path + i):
        x = i+'/'+j
        im_path.append(x)

In [ ]:
def get_window_value(feature):
    if type(feature) == pydicom.multival.MultiValue:
        return np.int(feature[0])
    else:
        return np.int(feature)

pixelspacing_r = []
pixelspacing_c = []
slice_thicknesses = []
ids = []
id_pth = []
row_values = []
column_values = []
window_widths = []
window_levels = []

for i in im_path:
    ids.append(i.split('/')[0]+'_'+i.split('/')[1])
    example_dcm = listdir(train_path  + i + "/")[0]
    id_pth.append(train_path + i)
    dataset = pydicom.dcmread(train_path + i + "/" + example_dcm)
    
    window_widths.append(get_window_value(dataset.WindowWidth))
    window_levels.append(get_window_value(dataset.WindowCenter))
    
    spacing = dataset.PixelSpacing
    slice_thicknesses.append(dataset.SliceThickness)
    
    row_values.append(dataset.Rows)
    column_values.append(dataset.Columns)
    pixelspacing_r.append(spacing[0])
    pixelspacing_c.append(spacing[1])
    
scan_properties = pd.DataFrame(data=ids, columns=["ID"])
scan_properties.loc[:, "rows"] = row_values
scan_properties.loc[:, "columns"] = column_values
scan_properties.loc[:, "area"] = scan_properties["rows"] * scan_properties["columns"]
scan_properties.loc[:, "pixelspacing_r"] = pixelspacing_r
scan_properties.loc[:, "pixelspacing_c"] = pixelspacing_c
scan_properties.loc[:, "pixelspacing_area"] = scan_properties.pixelspacing_r * scan_properties.pixelspacing_c
scan_properties.loc[:, "slice_thickness"] = slice_thicknesses
scan_properties.loc[:, "id_pth"] = id_pth
scan_properties.loc[:, "window_width"] = window_widths
scan_properties.loc[:, "window_level"] = window_levels
scan_properties.head().T

### pixelspacing(ピクセル間隔)
* dicom ファイルにある pixelspacing 属性は重要な属性です。これは、1つのピクセルがどのくらいの物理的な距離をカバーしているかを教えてくれます。横断スライスの平面内のX方向とY方向を記述する2つの値しかないことがわかります。
* 1人の患者の場合、このピクセル間隔は通常、すべてのスライスで同じです。
* しかし、患者の間では医師やクリニックの個人的または組織的な好みによってピクセル間隔が異なる場合があり、またスキャナーのタイプにも依存します。その結果、肺のサイズで2つの画像を比較した場合、自動的に大きい方が臓器の物理的なサイズが大きいことを意味するわけではありません。

In [ ]:
sns.set_style('darkgrid')
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(pixelspacing_r, ax=ax[0], color='green', kde_kws=dict(lw=3, ls="--",color='red'))
ax[0].set_title("Pixel spacing distribution \n in row direction ")
ax[0].set_ylabel("Counts in train")
ax[0].set_xlabel("mm")
sns.distplot(pixelspacing_c, ax=ax[1], color="Blue",kde_kws=dict(lw=3, ls="--",color='red'))
ax[1].set_title("Pixel spacing distribution \n in column direction");
ax[1].set_ylabel("Counts in train");
ax[1].set_xlabel("mm");

我々は、値が本当に患者から患者に多くの違いがあることを見ることができます! 彼らはmmで与えられているように、CTスキャンは通常、512行と列の値をカバーしています。

### 1回のCTスキャンでカバーされる物理的な領域とスライス量

さて、ct-scanでカバーされる物理的な距離を計算するためのいくつかの重要な量がわかりました!

In [ ]:
scan_properties["r_distance"] = scan_properties.pixelspacing_r * scan_properties.rows
scan_properties["c_distance"] = scan_properties.pixelspacing_c * scan_properties["columns"]
scan_properties["area_cm2"] = 0.1* scan_properties["r_distance"] * 0.1*scan_properties["c_distance"]
scan_properties["slice_volume_cm3"] = 0.1*scan_properties.slice_thickness * scan_properties.area_cm2

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(scan_properties.area_cm2, ax=ax[0], color="Limegreen",kde_kws=dict(lw=3, ls="--",color='red'))
sns.distplot(scan_properties.slice_volume_cm3, ax=ax[1], color="Mediumseagreen",kde_kws=dict(lw=3, ls="--",color='red'))
ax[0].set_title("CT-slice area in $cm^{2}$")
ax[1].set_title("CT-slice volume in $cm^{3}$")
ax[0].set_xlabel("$cm^{2}$")
ax[1].set_xlabel("$cm^{3}$");

In [ ]:
scan_properties.head(3).T

In [ ]:
scan_properties.describe().T

In [ ]:
scan_properties.to_csv('Pulmonary_Embolism_CT_scans_data.csv',index=False)

In [ ]:
scan_cols = scan_properties.copy()
scan_cols.drop(['rows','columns','area'],axis=1,inplace=True)

corr = scan_cols.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(10, 10))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=.8,cmap="viridis",annot=True)

異常なヒートマップが見られますが、これは値のほとんどが、いくつかの特徴を高度に相関させるようにして、得られた値だからです。

In [ ]:
cols = train.copy()
cols.drop(['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID'],axis=1,inplace=True)
columns = cols.columns

In [ ]:
fig, ax = plt.subplots(7,2,figsize=(16,28))
for i,col in enumerate(columns): 
    plt.subplot(7,2,i+1)
    sns.countplot(cols[col],palette='hot')   

In [ ]:
corr = cols.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(12, 12))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=.8,cmap="summer",annot=True)

# Acknowledgements
1. [Excellent work of Laura Fink](https://www.kaggle.com/allunia/pulmonary-fibrosis-dicom-preprocessing)
2. [Insights used by prk007](https://www.kaggle.com/prk007/insights-from-tabular-and-image-data)
3. [3D reconstruction by Md. Redwan Karim Sony](https://www.kaggle.com/redwankarimsony/rsna-str-3d-stacking-3d-plot-segmentation/comments)